In [ ]:
import csv
import requests
import json

In [ ]:
progress_id_files = '/path/to/csv/with/progress/ids'
giles_client_token = 'giles token'
github_token = 'personal github token'
output_folder = '/path/to/folder/for/saved/documents'


### Get a list of all progress ids

In [ ]:
progress_ids = {}
with open(progress_id_files) as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count > 0:
            if row['Resource'] not in progress_ids:
                progress_ids[row['Resource']] = row['Progress ID']
        line_count += 1
print("Downloading " + str(len(progress_ids)) + " files")

### Get a temporary Giles token

In [ ]:
headers = { 'Authorization': 'token ' + giles_client_token}
data = {'providerToken': github_token}
token_response = requests.post('https://diging.asu.edu/geco-giles/rest/token', headers=headers, data=data)
response_text = token_response.text
token = ''
if token_response.status_code == 200:
    text = json.loads(response_text)
    token = text["token"]
print("Using token: " + token)

In [ ]:
### Download files

In [ ]:
headers = { 'Authorization': 'token ' + token}

def downloadText(upload, file):
    if not 'extractedText' in upload:
        print("Skipped " + file)
        return
    
    if upload['extractedText']['url']:
        extracted_text = requests.get(upload['extractedText']['url'], headers=headers)
        with open(output_folder + file + ".txt", 'w') as f:
            f.write(extracted_text.text)
            print("Saved " + file + ".txt")
                    
for file in progress_ids:
    prog_id = progress_ids[file]
    if not prog_id:
        continue
    document = requests.get('https://diging.asu.edu/geco-giles/rest/files/upload/check/'+prog_id, headers=headers)
    upload_info = json.loads(document.text)
    
    # download the text
    for upload in upload_info:
        downloadText(upload, file)
